In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import numpy as np

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

In [7]:
# Rode esta célula APENAS UMA VEZ para criar o arquivo CSV com os dados

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Configurações do posto
precos = {'Gasolina': 6.29, 'Etanol': 4.56, 'Diesel': 7.89}
bombas = {
    'Gasolina': list(range(1,7)),
    'Etanol': list(range(7,13)),
    'Diesel': list(range(13,19))
}
frentistas_dia = ['Caio', 'Luciano', 'Pedro']
frentistas_noite = ['Luís', 'Tarcísio', 'João']

# Datas: 7 dias começando em 01/01/2023
start_date = datetime(2023, 1, 1)
dates = [start_date + timedelta(days=i) for i in range(7)]

# Lista para guardar todos os registros
data = []
np.random.seed(42)  # Para sempre gerar os mesmos dados

for date in dates:
    # Turno do dia (6h às 18h) - mais movimento
    num_day = np.random.randint(120, 180)
    for _ in range(num_day):
        hour = np.random.randint(6, 18)
        minute = np.random.randint(0, 60)
        ts = datetime(date.year, date.month, date.day, hour, minute)
        combustivel = np.random.choice(['Gasolina', 'Etanol', 'Diesel'])
        bomba_id = np.random.choice(bombas[combustivel])
        frentista = np.random.choice(frentistas_dia)
        litros = round(np.random.uniform(10, 100), 2)
        valor = round(litros * precos[combustivel], 2)
        data.append([ts, date.date(), bomba_id, combustivel, frentista, litros, valor])
    
    # Turno da noite (18h às 6h) - menos movimento
    num_night = np.random.randint(60, 100)
    for _ in range(num_night):
        if np.random.rand() > 0.3:  # Mais chance de ser após 18h
            hour = np.random.randint(18, 24)
            ts_date = date
        else:
            hour = np.random.randint(0, 6)
            ts_date = date + timedelta(days=1)
        minute = np.random.randint(0, 60)
        ts = datetime(ts_date.year, ts_date.month, ts_date.day, hour, minute)
        combustivel = np.random.choice(['Gasolina', 'Etanol', 'Diesel'])
        bomba_id = np.random.choice(bombas[combustivel])
        frentista = np.random.choice(frentistas_noite)
        litros = round(np.random.uniform(10, 100), 2)
        valor = round(litros * precos[combustivel], 2)
        data.append([ts, date.date(), bomba_id, combustivel, frentista, litros, valor])

# Criar o DataFrame
columns = ['Timestamp', 'Data', 'Bomba_ID', 'Tipo_Combustivel', 'Frentista', 'Litros_Vendidos', 'Valor_Total']
df = pd.DataFrame(data, columns=columns)
df = df.sort_values('Timestamp').reset_index(drop=True)

# Salvar o arquivo CSV na pasta
df.to_csv('vendas_posto.csv', index=False)

print(f"Dataset gerado com sucesso!")
print(f"Total de abastecimentos: {len(df)}")
print(f"Arquivo salvo: vendas_posto.csv")

Dataset gerado com sucesso!
Total de abastecimentos: 1648
Arquivo salvo: vendas_posto.csv


In [8]:
# Carrega o CSV (rode sempre que abrir o notebook)
df = pd.read_csv('vendas_posto.csv', parse_dates=['Timestamp', 'Data'])
df.head(10)

,Timestamp,Data,Bomba_ID,Tipo_Combustivel,Frentista,Litros_Vendidos,Valor_Total
0,2023-01-01 06:04:00,2023-01-01,3,Gasolina,Pedro,79.17,497.98
1,2023-01-01 06:08:00,2023-01-01,5,Gasolina,Caio,71.63,450.55
2,2023-01-01 06:08:00,2023-01-01,18,Diesel,Caio,87.25,688.40
3,2023-01-01 06:10:00,2023-01-01,2,Gasolina,Pedro,45.37,285.38
4,2023-01-01 06:12:00,2023-01-01,4,Gasolina,Luciano,81.03,509.68
5,2023-01-01 06:24:00,2023-01-01,13,Diesel,Caio,27.88,219.97
6,2023-01-01 06:25:00,2023-01-01,1,Gasolina,Pedro,26.61,167.38
7,2023-01-01 06:29:00,2023-01-01,2,Gasolina,Luciano,95.42,600.19
8,2023-01-01 06:32:00,2023-01-01,3,Gasolina,Pedro,94.73,595.85
9,2023-01-01 06:34:00,2023-01-01,6,Gasolina,Pedro,78.47,493.58


In [9]:
from ipywidgets import interact, widgets
import pandas as pd

# Garantir que a coluna 'Data' esteja no formato date (sem hora)
df['Data'] = pd.to_datetime(df['Data']).dt.date

def consulta_interativa(frentista='Todos', data_inicial=None, data_final=None, combustivel=None):
    df_filtrado = df.copy()
    
    # Filtro por frentista
    if frentista != 'Todos':
        df_filtrado = df_filtrado[df_filtrado['Frentista'] == frentista]
    
    # Filtro por intervalo de datas
    if data_inicial is not None:
        inicio = pd.to_datetime(data_inicial).date()
        df_filtrado = df_filtrado[df_filtrado['Data'] >= inicio]
    
    if data_final is not None:
        fim = pd.to_datetime(data_final).date()
        df_filtrado = df_filtrado[df_filtrado['Data'] <= fim]
    
    # Se nenhum dos dois foi selecionado, não aplica filtro de data
    # (já está mostrando todos os dias)
    
    # Filtro por combustível
    if combustivel and combustivel != 'Todos':
        df_filtrado = df_filtrado[df_filtrado['Tipo_Combustivel'] == combustivel]
    
    if df_filtrado.empty:
        print("Nenhuma venda encontrada com esses filtros.")
        return
    
    total_litros = df_filtrado['Litros_Vendidos'].sum().round(2)
    total_valor = df_filtrado['Valor_Total'].sum().round(2)
    abastecimentos = len(df_filtrado)
    
    # Descrição bonita do período
    if data_inicial and data_final and data_inicial != data_final:
        periodo = f"Período: {data_inicial} a {data_final}"
    elif data_inicial:
        periodo = f"Data: {data_inicial}"
    elif data_final:
        periodo = f"Data: {data_final}"
    else:
        periodo = "Período: Todos os dias"
    
    print(f"📊 Resultado da consulta:")
    print(f"   Frentista: {frentista}")
    print(f"   {periodo}")
    print(f"   Combustível: {combustivel or 'Todos'}")
    print(f"   Abastecimentos: {abastecimentos}")
    print(f"   Litros vendidos: {total_litros:,.2f} L")
    print(f"   Valor total: R$ {total_valor:,.2f}")

# Lista de datas disponíveis no formato string YYYY-MM-DD
datas_disponiveis = sorted(df['Data'].unique())
opcoes_datas = [None] + [d.strftime('%Y-%m-%d') for d in datas_disponiveis]

# Widgets interativos
interact(
    consulta_interativa,
    frentista=widgets.Dropdown(options=['Todos', 'Caio', 'Luciano', 'Pedro', 'Luís', 'Tarcísio', 'João'], value='Todos', description='Frentista:'),
    data_inicial=widgets.Dropdown(options=opcoes_datas, value=None, description='Data Inicial:'),
    data_final=widgets.Dropdown(options=opcoes_datas, value=None, description='Data Final:'),
    combustivel=widgets.Dropdown(options=['Todos', None, 'Gasolina', 'Etanol', 'Diesel'], value=None, description='Combustível:')
);

interactive(children=(Dropdown(description='Frentista:', options=('Todos', 'Caio', 'Luciano', 'Pedro', 'Luís',…

In [10]:
def grafico_vendas(frentista='Todos'):
    df_plot = df.copy()
    if frentista != 'Todos':
        df_plot = df_plot[df_plot['Frentista'] == frentista]
    
    vendas_diarias = df_plot.groupby('Data').agg({'Valor_Total': 'sum'}).reset_index()
    
    plt.figure(figsize=(10, 5))
    plt.plot(vendas_diarias['Data'], vendas_diarias['Valor_Total'], marker='o', linewidth=2)
    plt.title(f'Vendas Diárias em Valor (R$) - Frentista: {frentista}')
    plt.ylabel('Valor Total (R$)')
    plt.xlabel('Data')
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

interact(grafico_vendas, frentista=['Todos', 'Caio', 'Luciano', 'Pedro', 'Luís', 'Tarcísio', 'João']);

interactive(children=(Dropdown(description='frentista', options=('Todos', 'Caio', 'Luciano', 'Pedro', 'Luís', …